These are helper modules for python that help you to quickly setup and execute a series of first-principles calculations using, along with many others, VASP and ABINIT.

# "Quick" Intro to Python 2.7

Due to a constantly changing implementation, it is best practice to start python codes using more accurate code from later versions. In particular, the `print()` function and the division operator `/`. It is best to not even learn the old `print` behavior, but here are some examples of the explicit and implicit division rules.

In [ ]:
# Import new implementation of specific functions.
from __future__ import print_function, division
print('Hello') # Note: newline is automatically included
print(5/2)   # Implicit float division. Old division would produce `2`.
print(5./2)  # Explicit float division.
print(5//2)  # Explicit integer division.
print(5.//2) # Also integer division - i.e. "floor" function.

Python is lazy about variable allocation and types. Python infers the type from the context and easily combines similar types.

In [ ]:
# Add two different types
a = 5
b = 6.2
print(type(a))
print(type(b))
print(a+b) # 5 is automatically converted to 5.0

Variables can be lazily reassigned to another type:

In [ ]:
a = True
print(a)
print(a+b) # True is automatically converted into 1.0

Lists are ordered arrays that can contain anything:

In [ ]:
lst = [1,False,3.14,{},'Hello']
print(lst)

Lists can easily iterate:

In [ ]:
for item in lst:
    print(item,type(item))

Lists can also be accessed and modified like C arrays:

In [ ]:
print(lst[0])
print(lst[4])
lst[3] = 14.7
print(lst)
print(lst[5]) # Raises an IndexError because the list only has 5 elements.

Dictionaries are hashable (key, value) pairs.

In [ ]:
dct = {'a': 1}
dct['b'] = 2
print(dct)
print(dct['b'] + dct['a'])
print(dct['c']) # Raises a KeyError because 'c' is not a valid key.

Dicts have multiple ways of iterating over the contents depending on what you want. Also note that simple for-loops can be done in one line.

In [ ]:
for key in dct.keys(): print(key)
for value in dct.values(): print(value)
for key,value in dct.items(): print(key, ':', value)

In [ ]:
print(dct.keys())
print(dct.values())
print(dct.items())

Tuples are unchangable (immutable) lists.

In [ ]:
tpl = (1,2,3)
print(tpl)
print(tpl[0])
tpl[0] = 2 # Raises a TypeError because tuples cannot be changed.

Iterators are objects that act like lists, but they do not construct all of the items at once. The items are constructed as-needed.

In [ ]:
print(xrange(3)) # Prints "an object"
print(range(3)) # Prints a list

In [ ]:
for i in xrange(3): print(i)
for i in range(3): print(i)

Here are two definitions: a function that returns a list, and a generator that yields the list items. The `func_list` subroutine will generate the entire list and then return it. The `gen_list` subroutine will produce the first item, but then remember its position for when the next item is needed.

In [ ]:
def func_list():
    return [1,2,3]
print(func_list()) # Prints the complete list.

def gen_list():
    yield 1
    yield 2
    yield 3
print(gen_list()) # Prints an object. We can't see the results until we access them!

In [ ]:
for i in func_list(): print(i)
for i in gen_list(): print(i)

Generators and iterators are useful for situations where the entire list cannot easily be created all at once (too much memory) or where other factors determine the next item in the list (maybe it yields a time-dependent result from a network).

# Installing Pymatgen

There are many requirements for pymatgen. First, you need a python interpretor, and installations of numpy (numerical python) and a few other modules. I will not cover this aspect because there are many tutorials online. I am using Enthought Canopy because it is free for educational institutions. Once Canopy is installed, I load the Canopy Terminal "Tools > Canopy Terminal" and run "pip install pymatgen". If all succeedes, you will now have access to pymatgen in your python installation.

# Pymatgen interface to VASP

Pymatgen will allow you to programmatically create and read VASP input and output. Most of the time, we will manually create some input for VASP and then use pymatgen to perform some data analysis, so I will start with how to import VASP results.

In [29]:
%%bash
cat /Users/liam/Work/POSCAR

HgTe
6.21
0.00 0.50 0.50
0.50 0.00 0.50
0.50 0.50 0.00
Hg Te
1  1
Direct
0.00 0.00 0.00
0.25 0.25 0.25


In [30]:
from pymatgen.io.vaspio.vasp_output import Poscar

# Poscar is a python module that contains helper methods. `pos` is a POSCAR object that
# contains the information from the file.
pos = Poscar.from_file('/Users/liam/Work/POSCAR')

In [31]:
print(pos)

HgTe
1.0
0.000000 3.105000 3.105000
3.105000 0.000000 3.105000
3.105000 3.105000 0.000000
Hg Te
1 1
direct
0.000000 0.000000 0.000000 Hg
0.250000 0.250000 0.250000 Te



In [32]:
print(pos.structure) # The structure is a more general structure object

Structure Summary (Hg1 Te1)
Reduced Formula: HgTe
abc   :   4.391133   4.391133   4.391133
angles:  60.000000  60.000000  60.000000
Sites (2)
1 Hg     0.000000     0.000000     0.000000
2 Te     0.250000     0.250000     0.250000


In [33]:
print(pos.structure.species) # A list of species

[Element Hg, Element Te]


In [34]:
print(pos.structure.lattice_vectors()) # Produces a 3x3 list

[[ 0.     3.105  3.105]
 [ 3.105  0.     3.105]
 [ 3.105  3.105  0.   ]]


In [35]:
for site in pos.structure: print(site.frac_coords.tolist(), site.coords.tolist())

([0.0, 0.0, 0.0], [0.0, 0.0, 0.0])
([0.25, 0.25, 0.25], [1.5525, 1.5525, 1.5525])


In [36]:
print(pos.structure[0])

[ 0.  0.  0.] Hg


In [37]:
print(pos.structure[0].frac_coords.tolist())

[0.0, 0.0, 0.0]


In [38]:
# This won't work because the coords are "read-only" attributes.
pos.structure[0].coords = (0.,0.,0.)

AttributeError: can't set attribute

In [39]:
print(pos.structure[0], pos.structure[1])
tmp = pos.structure[0]
pos.structure[0] = pos.structure[1]
pos.structure[1] = tmp
print(pos.structure[0], pos.structure[1])

(PeriodicSite: Hg (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000], PeriodicSite: Te (1.5525, 1.5525, 1.5525) [0.2500, 0.2500, 0.2500])
(PeriodicSite: Te (1.5525, 1.5525, 1.5525) [0.2500, 0.2500, 0.2500], PeriodicSite: Hg (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000])


In [40]:
print(pos)

HgTe
1.0
0.000000 3.105000 3.105000
3.105000 0.000000 3.105000
3.105000 3.105000 0.000000
Te Hg
1 1
direct
0.250000 0.250000 0.250000 Te
0.000000 0.000000 0.000000 Hg

